In [14]:
# 🔧 Setup: Imports & Pfade
import sys
import os
sys.path.append("/home/karl-/liquidstatemachines")

import torch
import tonic.transforms as transforms
from data.dataloader import load_filtered_shd_dataloader
from models.sffnn_batched import Net


In [15]:
# 🖥️ Device konfigurieren
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [16]:
# 🧠 Modell-Definition (muss exakt wie beim Training sein)
model = Net(num_inputs=350, num_hidden=1000, num_outputs=10, num_steps=250, beta=0.9).to(device)


In [17]:
# 📦 Modell-Gewichte laden
model.load_state_dict(torch.load("../model_export/model_weights.pth", map_location=device))
model.eval()
print("Modell erfolgreich geladen.")


Modell erfolgreich geladen.


In [18]:
# 🔄 Testdaten vorbereiten
transform = transforms.Compose([
    transforms.Downsample(spatial_factor=0.5),
    transforms.ToFrame(sensor_size=(350,1,1), n_time_bins=250)
])

test_dataloader = load_filtered_shd_dataloader(label_range=range(10), transform=transform, train=False, batch_size=1)


In [19]:
model.to(device)
model.eval()
# 🧪 Eine Vorhersage machen
with torch.no_grad():
    for events, labels in test_dataloader:
        events = events.squeeze(2).to(device).float()  # [1, T, 1, 350] → [1, T, 350]
        labels = labels.to(device)

        spk_rec, _ = model(events)
        spike_sums = spk_rec.sum(dim=1)  # → [B, num_outputs]
        pred = torch.argmax(spike_sums, dim=1)

        print(f"🔍 Predicted: {pred.item()}, ✅ Ground Truth: {labels.item()}")
        break  # nur ein Sample testen


torch.Size([1, 250, 350])
🔍 Predicted: 7, ✅ Ground Truth: 7


In [ ]:
from utils.metrics import *
print_full_dataloader_accuracy_batched(model,test_dataloader)